In [2]:
import os
import numpy as np
import nibabel as nib
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from skimage import exposure

def apply_window_level(image, window_width, window_level):
    """
    根据给定的窗宽和窗位调整图像的灰度值。
    
    参数：
        image (numpy.ndarray): 原始图像数据。
        window_width (int): 窗宽。
        window_level (int): 窗位。
        
    返回：
        numpy.ndarray: 调整后的图像数据。
    """
    min_window = window_level - window_width // 2
    max_window = window_level + window_width // 2
    
    # 应用窗宽和窗位
    adjusted_image = np.clip(image, min_window, max_window)
    adjusted_image = ((adjusted_image - min_window) / (max_window - min_window)) * 255.0
    return adjusted_image.astype(np.uint8)

def calculate_gradient_based_window(image):
    """
    基于图像梯度计算窗宽和窗位。
    
    参数：
        image (numpy.ndarray): 原始图像数据。
        
    返回：
        tuple: 窗宽和窗位。
    """
    # 使用Sobel算子计算图像的梯度（考虑更多方向）
    sobel_x = ndi.sobel(image.astype(np.float32), axis=0)
    sobel_y = ndi.sobel(image.astype(np.float32), axis=1)
    sobel_z = ndi.sobel(image.astype(np.float32), axis=2)
    
    gradient = np.sqrt(sobel_x**2 + sobel_y**2 + sobel_z**2)
    
    # 计算梯度的均值和标准差
    mean_gradient = np.mean(gradient)
    std_gradient = np.std(gradient)
    
    # 根据梯度统计信息确定窗位和窗宽
    window_level = mean_gradient
    window_width = 4 * std_gradient  # 可以根据具体需求调整此系数
    
    return window_width, window_level

def enhance_contrast(image):
    """
    使用自适应直方图均衡化增强图像对比度。
    
    参数：
        image (numpy.ndarray): 原始图像数据。
        
    返回：
        numpy.ndarray: 对比度增强后的图像数据。
    """
    # 归一化图像到0到1之间
    image = image.astype(np.float32)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    
    # 自适应直方图均衡化
    enhanced_image = exposure.equalize_adapthist(image)
    return (enhanced_image * 255).astype(np.uint8)

def process_image_file(input_path, output_path):
    """
    处理单个 .nii.gz 文件并保存调整后的图像。
    
    参数：
        input_path (str): 输入 .nii.gz 文件的路径。
        output_path (str): 保存调整后的 .nii.gz 文件的路径。
    """
    # 加载NIfTI文件
    img = nib.load(input_path)
    image_data = img.get_fdata()

    # 基于梯度计算窗宽和窗位
    window_width, window_level = calculate_gradient_based_window(image_data)

    # 根据计算出的窗宽和窗位调整图像
    adjusted_image = apply_window_level(image_data, window_width, window_level)
    
    # 增强图像对比度
    enhanced_image = enhance_contrast(adjusted_image)

    # 将调整后的图像保存为新的NIfTI文件
    adjusted_img = nib.Nifti1Image(enhanced_image, img.affine, img.header)
    nib.save(adjusted_img, output_path)

def batch_process_images(input_folder, output_folder):
    """
    批量处理输入文件夹中的所有 .nii.gz 文件，并保存到输出文件夹。
    
    参数：
        input_folder (str): 包含 .nii.gz 文件的输入文件夹路径。
        output_folder (str): 保存调整后 .nii.gz 文件的输出文件夹路径。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.nii.gz'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            process_image_file(input_path, output_path)
            print(f'已处理 {filename}')

def process_image_file(input_path, output_path):
    try:
        img = nib.load(input_path)
        image_data = img.get_fdata()
        window_width, window_level = calculate_gradient_based_window(image_data)
        adjusted_image = apply_window_level(image_data, window_width, window_level)
        enhanced_image = enhance_contrast(adjusted_image)
        adjusted_img = nib.Nifti1Image(enhanced_image, img.affine, img.header)
        nib.save(adjusted_img, output_path)
        print(f'已处理 {input_path}')
    except Exception as e:
        print(f'处理 {input_path} 时出错: {e}')

            
# 设置输入和输出文件夹路径
input_folder = r'G:\My computer backup\My paper\Breast cancer\CW\NII\B_in'  # 替换为实际输入文件夹路径
output_folder = r'G:\My computer backup\My paper\Breast cancer\CW\NII\CW_G'  # 替换为实际输出文件夹路径

# 批量处理图像
batch_process_images(input_folder, output_folder)


已处理 G:\My computer backup\My paper\Breast cancer\CW\NII\B_in\FC050_1.nii.gz
已处理 FC050_1.nii.gz
已处理 G:\My computer backup\My paper\Breast cancer\CW\NII\B_in\FC050_3.nii.gz
已处理 FC050_3.nii.gz
